In [1]:
import pandas as pd

In [5]:
df = pd.read_csv('train.csv')
df.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


In [6]:
def determine_winner(row):
    if row['winner_tie'] == 1:
        return 0
    elif row['winner_model_a'] == 1:
        return 1
    elif row['winner_model_b'] == 1:
        return 2
    else:
        return -1

df['winner'] = df.apply(determine_winner, axis=1)

df = df.drop(columns=['id', 'winner_model_a', 'winner_model_b', 'winner_tie'])
df.head()

,model_a,model_b,prompt,response_a,response_b,winner
0,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1
1,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",2
2,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0
3,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1
4,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",2


In [7]:
import re
import ast
import html

def extract_valuable_sentence(raw_prompt):
    try:
        text = ast.literal_eval(raw_prompt)[0]
    except:
        text = str(raw_prompt)

    text = html.unescape(text)
    text_clean = re.sub(r'\\u[0-9a-fA-F]{4}', '', text)
    text_clean = re.sub(r'<.*?>', '', text_clean)
    text_clean = re.sub(r'\$.*?\$', '', text_clean)
    text_clean = re.sub(r'\\[a-zA-Z]+\b', '', text_clean)
    valuable = re.findall(r'[A-Za-z0-9,.\-()\s]+', text_clean)
    valuable_text = ''.join(valuable).strip()
    valuable_text = re.sub(r'\s+', ' ', valuable_text).strip()

    return valuable_text

df['prompt_valuable'] = df['prompt'].apply(extract_valuable_sentence)
df['response_a_cleaned'] = df['response_a'].apply(extract_valuable_sentence)
df['response_b_cleaned'] = df['response_b'].apply(extract_valuable_sentence)

In [8]:
df.head()

,model_a,model_b,prompt,response_a,response_b,winner,prompt_valuable,response_a_cleaned,response_b_cleaned
0,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I dont have personal beliefs or opin..."
1,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",2,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...
2,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,explain function calling. how would you call a...,Function calling is the process of invoking or...,Function calling is the process of invoking a ...
3,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,How can I create a test set for a very rare ca...,Creating a test set for a very rare category c...,When building a classifier for a very rare cat...
4,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",2,What is the best way to travel from Tel-Aviv t...,The best way to travel from Tel Aviv to Jerusa...,The best way to travel from Tel-Aviv to Jerusa...


In [9]:
df.to_csv("cleanedn.csv")

In [10]:
!pip install -q transformers torch

from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.6 MB/s eta 0:00:00


In [11]:
pip install nltk

In [12]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [13]:
df_nan = pd.read_csv('cleanedn.csv', index_col=0)
df_new = df_nan.dropna().copy()
df_new.head()

,model_a,model_b,prompt,response_a,response_b,winner,prompt_valuable,response_a_cleaned,response_b_cleaned
0,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I dont have personal beliefs or opin..."
1,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",2,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...
2,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,explain function calling. how would you call a...,Function calling is the process of invoking or...,Function calling is the process of invoking a ...
3,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,How can I create a test set for a very rare ca...,Creating a test set for a very rare category c...,When building a classifier for a very rare cat...
4,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",2,What is the best way to travel from Tel-Aviv t...,The best way to travel from Tel Aviv to Jerusa...,The best way to travel from Tel-Aviv to Jerusa...


In [14]:
df_new.shape

(57240, 9)

In [21]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

def get_sentiment_scores(text):
    scores = sid.polarity_scores(text)
    return scores['compound']

df_new_clean['sentiment_score_response_a'] = df_new_clean['response_a_cleaned'].fillna('').apply(get_sentiment_scores)
df_new_clean['sentiment_score_response_b'] = df_new_clean['response_b_cleaned'].fillna('').apply(get_sentiment_scores)

In [16]:
df.shape

(57477, 9)

In [22]:
df_new.head()

,model_a,model_b,prompt,response_a,response_b,winner,prompt_valuable,response_a_cleaned,response_b_cleaned,sentiment_score,sentiment_score_1,sentiment_score_2
0,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I dont have personal beliefs or opin...",0.2732,0.9883,0.7951
1,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",2,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,0.0000,0.6908,0.9231
2,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,explain function calling. how would you call a...,Function calling is the process of invoking or...,Function calling is the process of invoking a ...,0.0000,0.1280,-0.1027
3,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,How can I create a test set for a very rare ca...,Creating a test set for a very rare category c...,When building a classifier for a very rare cat...,0.3947,0.9868,0.9792
4,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",2,What is the best way to travel from Tel-Aviv t...,The best way to travel from Tel Aviv to Jerusa...,The best way to travel from Tel-Aviv to Jerusa...,0.6369,0.9702,0.8519


In [18]:
df_new.to_csv("Extraclean.csv")

In [6]:
df_new_1 = pd.read_csv('cosine.csv', index_col=0)

In [7]:
df_new_clean = pd.read_csv('cosine.csv', index_col=0)
df_new_clean = df_new_1.dropna().copy()
df_new_clean.head()

,model_a,model_b,prompt,response_a,response_b,winner,prompt_valuable,response_a_cleaned,response_b_cleaned,prompt_tokens,response_a_tokens,response_b_tokens,response_a_vector,response_b_vector,similarity
0,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I dont have personal beliefs or opin...","['is', 'it', 'morally', 'right', 'to', 'try', ...","['the', 'question', 'of', 'whether', 'it', 'is...","['as', 'an', 'ai', 'dont', 'have', 'personal',...",[-1.28138125e+00 2.72807896e-01 -7.93284774e-...,[-8.2122421e-01 -5.5793330e-02 9.1366239e-02 ...,0.496171
1,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",2,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,"['what', 'is', 'the', 'difference', 'between',...","['marriage', 'license', 'is', 'legal', 'docume...","['marriage', 'license', 'and', 'marriage', 'ce...",[-0.10447463 0.36869672 1.0076132 -0.092721...,[ 0.16412653 1.8182849 -0.16929767 0.411663...,0.638639
2,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,explain function calling. how would you call a...,Function calling is the process of invoking or...,Function calling is the process of invoking a ...,"['explain', 'function', 'calling', 'how', 'wou...","['function', 'calling', 'is', 'the', 'process'...","['function', 'calling', 'is', 'the', 'process'...",[-0.55253726 1.5453577 0.32660076 -0.060469...,[-0.20759338 1.6251178 1.203351 0.280565...,0.614142
3,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,How can I create a test set for a very rare ca...,Creating a test set for a very rare category c...,When building a classifier for a very rare cat...,"['how', 'can', 'create', 'test', 'set', 'for',...","['creating', 'test', 'set', 'for', 'very', 'ra...","['when', 'building', 'classifier', 'for', 'ver...",[-2.1498358 -1.9901431 2.6460633 0.621134...,[-2.1913319 -0.43704936 0.9436316 -0.948851...,0.731076
4,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",2,What is the best way to travel from Tel-Aviv t...,The best way to travel from Tel Aviv to Jerusa...,The best way to travel from Tel-Aviv to Jerusa...,"['what', 'is', 'the', 'best', 'way', 'to', 'tr...","['the', 'best', 'way', 'to', 'travel', 'from',...","['the', 'best', 'way', 'to', 'travel', 'from',...",[-0.83134377 0.27480534 0.5567854 -0.398010...,[-5.9789598e-01 5.4763108e-01 4.4284627e-01 ...,0.636448


In [17]:
df_new_clean.shape

(57240, 23)

In [18]:
import re

def get_verbosity_features(text):
    words = len(text.split())

    # Rough sentence split based on punctuation
    sentences = re.split(r'[.!?]+', text)
    num_sentences = len([s for s in sentences if s.strip()])
    avg_sentence_len = round(words / (num_sentences or 1), 3)

    return words, avg_sentence_len

In [19]:
df_new_clean[['resp_a_words','resp_a_avg_sent_len']] = \
    df_new_clean['response_a_cleaned'].apply(lambda x: pd.Series(get_verbosity_features(x)))

df_new_clean[['resp_b_words', 'resp_b_avg_sent_len']] = \
    df_new_clean['response_b_cleaned'].apply(lambda x: pd.Series(get_verbosity_features(x)))

In [20]:
df_new_clean['verbosity_diff'] = df_new_clean['resp_a_words'] - df_new_clean['resp_b_words']

In [21]:
df_new_clean['prompt_valuable_words'] = df_new_clean['prompt_valuable'].apply(lambda x: len(x.split()))
df_new_clean['a_to_prompt_valuable_ratio'] = (df_new_clean['resp_a_words'] / (df_new_clean['prompt_valuable_words'] + 1)).round(3)
df_new_clean['b_to_prompt_valuable_ratio'] = (df_new_clean['resp_b_words'] / (df_new_clean['prompt_valuable_words'] + 1)).round(3)

In [22]:
# --- Function to count self-references (I, me, my, we, our, etc.)
def count_self_refs(text):
    return len(re.findall(r'\b(I|me|my|mine|we|our|us)\b', text, flags=re.IGNORECASE))

# --- Function to count self-promotional phrases
def count_self_promos(text):
    promo_patterns = [
        r'\bAs an AI\b', r'\bI can help\b', r'\bI recommend\b', r'\bI suggest\b',
        r'\bI am here to assist\b', r'\bYou can ask me\b'
    ]
    return sum(len(re.findall(p, text, flags=re.IGNORECASE)) for p in promo_patterns)

# --- Function to calculate first-person word ratio
def first_person_ratio(text):
    tokens = text.split()
    fp_words = {"i", "me", "my", "mine", "we", "our", "us"}
    fp_count = sum(1 for w in tokens if w.lower() in fp_words)
    return round(fp_count / (len(tokens) or 1),3)

# Self-reference counts
df_new_clean['resp_a_self_refs'] = df_new_clean['response_a_cleaned'].apply(count_self_refs)
df_new_clean['resp_b_self_refs'] = df_new_clean['response_b_cleaned'].apply(count_self_refs)
df_new_clean['self_ref_diff'] = df_new_clean['resp_a_self_refs'] - df_new_clean['resp_b_self_refs']

# Self-promotion counts
df_new_clean['resp_a_promos'] = df_new_clean['response_a_cleaned'].apply(count_self_promos)
df_new_clean['resp_b_promos'] = df_new_clean['response_b_cleaned'].apply(count_self_promos)
df_new_clean['self_promo_diff'] = df_new_clean['resp_a_promos'] - df_new_clean['resp_b_promos']

# First-person ratio
df_new_clean['resp_a_fp_ratio'] = df_new_clean['response_a_cleaned'].apply(first_person_ratio)
df_new_clean['resp_b_fp_ratio'] = df_new_clean['response_b_cleaned'].apply(first_person_ratio)
df_new_clean['fp_ratio_diff'] = df_new_clean['resp_a_fp_ratio'] - df_new_clean['resp_b_fp_ratio']

In [23]:
df_new_clean.drop(columns=[
    'resp_a_self_refs', 'resp_b_self_refs',
    'resp_a_promos', 'resp_b_promos',
    'resp_a_fp_ratio', 'resp_b_fp_ratio', 'resp_a_words','resp_b_words','prompt_valuable_words'
], inplace=True)

In [24]:
def categorize_prompt_valuable_simple(prompt_valuable):
    prompt_valuable = prompt_valuable.lower().strip()

    # Grouped: code + math → computation
    if re.search(r'\b(function|code|algorithm|python|java|c\+\+|write.*program|vlang|bug|debug|'
                 r'solve|equation|calculate|math|add|subtract|integral|derivative)\b', prompt_valuable):
        return 'computation'

    # Grouped: definition + reasoning → qa
    elif re.search(r'\b(what is|define|describe|meaning of|explain|why|how|compare|difference between|advantage of)\b', prompt_valuable):
        return 'qa'

    # Writing → creation
    elif re.search(r'\b(write.*(story|poem|essay|letter)|generate.*text|creative)\b', prompt_valuable):
        return 'creation'

    # Anything else
    else:
        return 'general'

In [25]:
df_new_clean['prompt_valuable_category'] = df_new_clean['prompt_valuable'].apply(categorize_prompt_valuable_simple)

In [26]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

def get_sentiment_scores(text):
    scores = sid.polarity_scores(text)
    return scores['compound']

df_new_clean['sentiment_score_response_a'] = df_new_clean['response_a_cleaned'].fillna('').apply(get_sentiment_scores)
df_new_clean['sentiment_score_response_b'] = df_new_clean['response_b_cleaned'].fillna('').apply(get_sentiment_scores)

In [15]:
df_new_clean.head()

,model_a,model_b,prompt,response_a,response_b,winner,prompt_valuable,response_a_cleaned,response_b_cleaned,prompt_tokens,...,response_b_tokens,response_a_vector,response_b_vector,similarity,resp_a_avg_sent_len,resp_b_avg_sent_len,verbosity_diff,self_ref_diff,self_promo_diff,fp_ratio_diff
0,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I dont have personal beliefs or opin...","['is', 'it', 'morally', 'right', 'to', 'try', ...",...,"['as', 'an', 'ai', 'dont', 'have', 'personal',...",[-1.28138125e+00 2.72807896e-01 -7.93284774e-...,[-8.2122421e-01 -5.5793330e-02 9.1366239e-02 ...,0.496171,16.400000,15.692308,452.0,-4,-2,-0.019608
1,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",2,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,"['what', 'is', 'the', 'difference', 'between',...",...,"['marriage', 'license', 'and', 'marriage', 'ce...",[-0.10447463 0.36869672 1.0076132 -0.092721...,[ 0.16412653 1.8182849 -0.16929767 0.411663...,0.638639,25.285714,15.432432,-40.0,-1,0,0.000000
2,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,explain function calling. how would you call a...,Function calling is the process of invoking or...,Function calling is the process of invoking a ...,"['explain', 'function', 'calling', 'how', 'wou...",...,"['function', 'calling', 'is', 'the', 'process'...",[-0.55253726 1.5453577 0.32660076 -0.060469...,[-0.20759338 1.6251178 1.203351 0.280565...,0.614142,13.800000,17.500000,-142.0,-2,0,-0.003468
3,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,How can I create a test set for a very rare ca...,Creating a test set for a very rare category c...,When building a classifier for a very rare cat...,"['how', 'can', 'create', 'test', 'set', 'for',...",...,"['when', 'building', 'classifier', 'for', 'ver...",[-2.1498358 -1.9901431 2.6460633 0.621134...,[-2.1913319 -0.43704936 0.9436316 -0.948851...,0.731076,17.866667,13.947368,271.0,0,0,0.000000
4,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",2,What is the best way to travel from Tel-Aviv t...,The best way to travel from Tel Aviv to Jerusa...,The best way to travel from Tel-Aviv to Jerusa...,"['what', 'is', 'the', 'best', 'way', 'to', 'tr...",...,"['the', 'best', 'way', 'to', 'travel', 'from',...",[-0.83134377 0.27480534 0.5567854 -0.398010...,[-5.9789598e-01 5.4763108e-01 4.4284627e-01 ...,0.636448,20.909091,13.555556,108.0,1,1,0.004348
